# Additional sql- Chimpanzee

#### The Answers located underneath the relevant code block. 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
db_config = {XXXXXXXXXXXX}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                                                             db_config['pwd'], 
                                                                                             db_config['host'], 
                                                                                             db_config['port'], 
                                                                                             db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})   

In [3]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [4]:
engine.table_names()

['item', 'rep_sales']

In [5]:
#summing the total sales
query = ''' 
              SELECT
              SUM("Total") AS total_sales
              FROM
              rep_sales;
              
        '''
queryResult(query)

,total_sales
0,28867.97


1) The overall Income in the data is 28867.97$

In [6]:
#grouping by sales rep and summing income
query = ''' 
              SELECT
              "Rep",
              SUM("Total") AS total_sales
              FROM
              rep_sales
              GROUP BY
              "Rep"
              ORDER BY
              total_sales DESC;
              
        '''
queryResult(query)

,Rep,total_sales
0,Kivell,3554.23
1,Thompson,3060.23
2,Jardine,3006.80
3,Jones,2969.49
4,Morgan,2669.98
5,Parent,2365.37
6,Gill,2124.21
7,Smith,1943.61
8,Howard,1941.51
9,Sorvino,1922.65


2) The sell Rep that brought the most income is Kivell (with 3554.23 $)

In [7]:
#checking which item brought the biggest sum of total income(ordering the results by descending)
query = ''' 
              SELECT
              "Item",
              SUM("Total") AS total_sales
              FROM
              rep_sales
              GROUP BY
              "Item"
              Order BY
              "total_sales" DESC;
              
        '''
queryResult(query)

,Item,total_sales
0,Binder,8229.12
1,Marker Set,6618.71
2,Pen Set,4683.66
3,Desk,4227.00
4,Pen,2769.02
5,Pencil,2340.46


3) The item that brought the most income isn a Binder (with 8229.12 $)

In [8]:
#Counting the number of items sold per region, filtering for desk so we only see how many dwsks solds and then ordering in descending order
query = ''' 
              SELECT
              "Region",
              SUM("Units") AS desks_sold
              FROM
              rep_sales
              WHERE
              "Item" LIKE '%Desk%'
              GROUP BY
              "Region"
              ORDER BY
              desks_sold DESC;
              
              
              
        '''
queryResult(query)

,Region,desks_sold
0,West,9
1,East,7
2,Central,7


4) The region that sold the most desks is west (with 9 desks sold)

In [9]:
#avraging the unit cost of pen for every sales rep and printing the result
query = ''' 
              SELECT
              "Rep",
              AVG("Unit_Cost") OVER (PARTITION BY "Units") AS average_pen_price
              FROM
              rep_sales
              WHERE
              "Item" LIKE '%Pen%'
              GROUP BY
              "Rep",
              "Unit_Cost",
              "Units"
              ORDER BY
              average_pen_price DESC;
              
              
              
        '''
queryResult(query)

,Rep,average_pen_price
0,Gill,19.99
1,Parent,19.99
2,Kivell,17.00
3,Jones,15.99
4,Parent,12.99
5,Morgan,12.49
6,Jones,8.99
7,Morgan,7.25
8,Thompson,6.99
9,Jones,6.99


5) The sells rep that sold the highest average price for pens was Gill (with avrage pen price of 19.99$)

In [10]:
#summing the number of units every rep sold between the first of April and 30th September (it seems that there isn't 31th september in the data although I found in google that There was 31th september in 2020)
query = ''' 
         SELECT
         "Rep",
         SUM("Units") AS units_sold
         FROM
         rep_sales
         WHERE
         "OrderDate"::date BETWEEN '01-04-2020' AND '30-09-2020'
         GROUP BY
         "Rep"
         ORDER BY
         units_sold DESC;
              
              
              
        '''
queryResult(query)

,Rep,units_sold
0,Andrews,309
1,Gill,256
2,Thompson,245
3,Howard,99
4,Kivell,94
5,Jones,86
6,Sorvino,79
7,Morgan,55
8,Joe,38


6) The sell rep that sold the most units of all the products between april and september 2020 is Andrews with 309 products. 

In [11]:
# substruct of the minimum unit cost from the minimum price, grouped by reps, filterd by the item pen set and orederd by the result of the substruction in descending order
query = ''' 
         SELECT
         rep_sales."Rep",
         (MAX(rep_sales."Unit_Cost"::Real) - (item."MinPrice"::real)) AS min_diffrence
         FROM
         rep_sales
         INNER JOIN item ON rep_sales."Item" = item."Item"
         WHERE
         rep_sales."Item" LIKE '%Pen Set%'
         GROUP BY
         "Rep",
         item."MinPrice"
         ORDER BY
         min_diffrence DESC;
         
              
              
              
        '''
queryResult(query)

,Rep,min_diffrence
0,Kivell,12.01
1,Jones,11.00
2,Parent,8.00
3,Morgan,7.50
4,Thompson,4.00
5,Jardine,0.00


7) The sales rep that sold the item pen set with the greatest price diffrence from the minimum price is Kivell with a pen set sold 12$ above the min price

In [12]:
# dividing the sales with the minimum price and converting to percentage, joining the two tables, filtering by units sold below the min price, grouping by rep and product, ordering the results. 
query = ''' 
         SELECT
         rep_sales."Rep",
         rep_sales."Item",
         ((((rep_sales."Unit_Cost"::Real) / (item."MinPrice"::real))-1)*-100) AS percent_diffrence
         FROM
         rep_sales
         INNER JOIN item ON rep_sales."Item" = item."Item"
         WHERE
         ((rep_sales."Unit_Cost"::Real) - (item."MinPrice"::real)) < 0 
         GROUP BY
         rep_sales."Rep",
         rep_sales."Item",
         rep_sales."Unit_Cost",
         item."MinPrice"
         ORDER BY
         percent_diffrence DESC;
         
         '''
queryResult(query)       

,Rep,Item,percent_diffrence
0,Gill,Pen,14.857143
1,Andrews,Pen,14.285713
2,Thompson,Pen,9.142858


8) Gill, Andrews and Thompson all sold items beneath the minimum price (14.8%, 14.2% and 9.14% lower), all the items were pens. 

In [13]:
query = ''' 
         SELECT
         DISTINCT rep_sales."Region" AS "Region",
         (100* SUM("Unit_Cost" - "MinPrice") OVER (PARTITION BY "Region") / SUM("Unit_Cost" - "MinPrice") OVER ())
         AS diffrence
         FROM
         rep_sales
         INNER JOIN item ON rep_sales."Item" = item."Item"
         WHERE
         "Unit_Cost" > "MinPrice"
         ORDER BY
         diffrence DESC;
         
         '''
queryResult(query)  

,Region,diffrence
0,West,40.747513
1,East,39.321534
2,Central,19.930953


 9) The region that had the highest share of price diffrence between the minimum price and the price sold is West with 40.74% diffrence.

In [14]:
#printing the name of the rep and the sum of the totals sales * 0.1 (10% percent of total sales), filtering only by item sold above minimum price, oredering by descending comission
query = ''' 
         SELECT
         rep_sales."Rep",
         (SUM(DISTINCT rep_sales."Total") *0.1) AS comission
         FROM
         rep_sales
         INNER JOIN item ON rep_sales."Item" = item."Item"
         WHERE
         ((rep_sales."Unit_Cost"::Real) - (item."MinPrice"::real)) > 0
         GROUP BY 
         rep_sales."Rep"
         ORDER BY
         comission DESC
         LIMIT 3;
         '''
queryResult(query)  

,Rep,comission
0,Jardine,275.730
1,Thompson,272.633
2,Morgan,266.998


10)The top 3 comissions were paid to Jardine(275.730),Thompson(272.633$) and Morgan(266.998f$)